# Install LlamaIndex and packages

In [ ]:
!pip install llama-index llama-index-tools-tavily-research -q

In [ ]:
!llamaindex-cli download-llamapack CorrectiveRAGPack --download-dir ./corrective_rag_pack -q

In [29]:
#or
# from llama_index.core.llama_pack import download_llama_pack

# # download and install dependencies
# CorrectiveRAGPack = download_llama_pack(
#     "CorrectiveRAGPack", "./corrective_rag_pack"
# )

# Get API keys

In [5]:
from google.colab import userdata
import os
tavily_ai_api_key = userdata.get('TAVILY_API_KEY')
LLAMAPARSE_API_KEY = userdata.get('LLAMAPARSE_API_KEY')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# LlamaParse

without gpt-4o : gpt4o_mode=False

In [ ]:
# parser = LlamaParse(api_key=LLAMAPARSE_API_KEY, gpt4o_mode=True,
#     result_type="markdown"  # "markdown" and "text" are available
# )

In [3]:
from google.colab import drive
drive.mount('/content/drive')

path='/content/drive/MyDrive/000_online_course/RAG/data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
path_file=path+'jpmorgan_annualreport-2022-splitted.pdf' #local path

from llama_parse import LlamaParse
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
import nest_asyncio;
nest_asyncio.apply()

# set up parser
parser = LlamaParse(api_key=LLAMAPARSE_API_KEY,
    result_type="markdown"  # "markdown" and "text" are available
)

# use SimpleDirectoryReader to parse our file
file_extractor = {".pdf": parser}
documents = SimpleDirectoryReader(input_files=[path_file], file_extractor=file_extractor).load_data()

Started parsing the file under job_id 1d6ed45d-26ea-4e9b-8901-ee4f3f372a03


### Some Thoughts

*   As you can see the chart part is not well parsed, because I didn't use the GPT-4o in LlamaParse.
*   I dind't in purpose to see if the CRAG can collect the adequate info from the web search.

In [5]:
print(documents[0].text)
#As you can see the chart part is not well parsed, because I didn't use the GPT-4o in LlamaParse.
#I dind't in purpose to see if the CRAG can collect the adequate info from the web search.

# Earnings, Diluted Earnings per Share and Return on Tangible Common Equity

$48.3 2004–2022 ($ in billions, except per share and ratio data)

| |Reported|Reported|Reported|Excluding reserve release/build|Excluding reserve release/build|Excluding reserve release/build|
|---|---|---|
| |2020|2021|2022|2020|2021|2022|
|Net income ($B)|$29.1|$48.3|$37.7|$38.4|$39.1|$40.4|
|Diluted EPS ($)|$8.88|$15.36|$12.09|$11.87|$12.35|$12.99|
|ROTCE|14.4%|23.0%|17.7%|19.3%|18.5%|19.1%|

Adjusted net income2 $32.5 $15.36 $10.72 $12.09 $26.9 $29.1 $9.00 $24.4 $24.7 $24.4 $21.7 $8.88 23% $21.3 18% $19.0 $17.4 $17.9 $6.00 $6.19 19% $6.31 $15.4 11% 17% $14.4 15% $14.4 15% 15% 13% $5.19 $5.29 12% $11.7 10% $4.48 $4.34 $8.5 $4.00 $4.33 6% $3.96 $2.35 $5.6 $2.26 $4.5 $1.35 $1.52

2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022

Net income Diluted earnings per share (EPS) Return on tangible common equity (ROTCE) Adjusted ROTCE2 ROTCE excluding reserve release/buil

# Modifying the base CorrectiveRAGPack

I made small modification to see some intermediate results,  like:

*   relevancy_score (the score the LLM gives to each retrieved document). This score is designed by this template prompt: DEFAULT_RELEVANCY_PROMPT_TEMPLATE

  - Assign a binary score to indicate the document's relevance.
  - Use 'yes' if the document is relevant to the question, or 'no' if it is not.

*   relevant_text : Only text with "yes" score

*   search_text ( coming from tavily)
*   query transformed by the LLM to enhance its search performance
*   I modified to use GPT-3.5-Turbo instead of GPT-4 specified by default.

Loading the modified CorrectiveRAGPack

In [7]:
from corrective_rag_pack.llama_index.packs.corrective_rag.base import CorrectiveRAGPack
corrective_rag = CorrectiveRAGPack(documents, tavily_ai_api_key)

## Question 1

Both values are in the document:

**Thoughts: **
Values well extracted and the calculation is correct

In [13]:
prompt = "What is the change in net income from 2021 to 2022?"
response = corrective_rag.run(prompt, similarity_top_k=2)

# ***************relevancy_results***************
# ['yes', 'no']



# ***************relevant_text***************
# # Earnings, Diluted Earnings per Share and Return on Tangible Common Equity

# $48.3 2004–2022 ($ in billions, except per share and ratio data)

# *********transformed_query_str for search_result********
# Net income difference between 2021 and 2022



# ***************search_text (from Tavily): first [:500]***************
# The Internal Revenue Service has released 2022 inflation adjustments for federal income tax brackets

***************relevancy_results***************
['yes', 'no']



***************relevant_text***************
# Earnings, Diluted Earnings per Share and Return on Tangible Common Equity

$48.3 2004–2022 ($ in billions, except per share and ratio data)

| |Reported|Reported|Reported|Excluding reserve release/build|Excluding reserve release/build|Excluding reserve release/build|
|---|---|---|
| |2020|2021|2022|2020|2021|2022|
|Net income ($B)|$29.1|$48.3|$37.7|$38.4|$39.1|$40.4|
|Diluted EPS ($)|$8.88|$15.36|$12.09|$11.87|$12.35|$12.99|
|ROTCE|14.4%|23.0%|17.7%|19.3%|18.5%|19.1%|

Adjusted net income2 $32.5 $15.36 $10.72 $12.09 $26.9 $29.1 $9.00 $24.4 $24.7 $24.4 $21.7 $8.88 23% $21.3 18% $19.0 $17.4 $17.9 $6.00 $6.19 19% $6.31 $15.4 11% 17% $14.4 15% $14.4 15% 15% 13% $5.19 $5.29 12% $11.7 10% $4.48 $4.34 $8.5 $4.00 $4.33 6% $3.96 $2.35 $5.6 $2.26 $4.5 $1.35 $1.52

2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022

Net income Diluted earnings 

In [14]:
print(response.response)

The change in net income from 2021 to 2022 is a decrease of $10.6 billion.


## Question 2

The same here:

**Thoughts:**
Values well extracted and the calculation is correct



In [ ]:
prompt = "Answer step-by-step to these questions: 1- What is the net income in 2021? 2-What is the net income in 2022? -What is the change in the net income from 2021 to 2022?"
response = corrective_rag.run(prompt, similarity_top_k=2)

# ***************relevancy_results***************
# ['yes', 'no']



# ***************relevant_text***************

# *********transformed_query_str for search_result********
# Net income 2021 and 2022 comparison.

In [16]:
print(response.response)

1. The net income in 2021 was $48.3 billion.
2. The net income in 2022 was $37.7 billion.
3. The change in net income from 2021 to 2022 was a decrease of $10.6 billion.


## Question 3

This information is not the report. So we expect the CRAG uses tavily search .

The transformed query by the CRAG: ==> *"Describe JPMorgan's activities."*

In [ ]:
prompt = "Give a description about JPMorgan activities?"
response = corrective_rag.run(prompt, similarity_top_k=2)
response

**Thoughts**: I'm expecting more valuable information, howver I get a quick description:

In [18]:
#The answer not satisfasante:
print(response.response)

JPMorgan Chase & Co. is a leading global financial services firm with assets of $2.6 trillion and operations worldwide. The company serves millions of customers in the United States and many of the world's most prominent corporate, institutional, and government clients under its J.P. Morgan and Chase brands.


## Question 4

I asked for Net Income in 2019. This value is not accuratly parsed. So we expect the CRAG to have a look at it on the internet:

**Thoughts**
Important to know: the quality of your question here is crucial. As the CRAG will transform your question to a better prompt to enhance its search perfomance, you need to give it all the important keywords:

For example:
*   You are attempted to ask this following question because you are chating with your llm:
**"What was the net income in 2019?"**

So the transformed query by the CRAG will be:
**"2019 net income"**

==> This when sent to tavily search, it will results in a poor quality answers!!

*   However if you ask:

**"What was the net income in 2019 of JPMorgan Chase bank?"**
The transformer question will be: **"JPMorgan Chase bank 2019 net income"**
=> This will result in a good answer.

In [19]:
prompt = "What was the net income in 2019 of JPMorgan Chase bank?"
response = corrective_rag.run(prompt, similarity_top_k=3)
response

***************relevancy_results***************
['no', 'no']



***************relevant_text***************




*********transformed_query_str for search_result********
JPMorgan Chase bank 2019 net income



***************search_text (from Tavily): first [:500]***************
Fiscal year is January-December. All values USD Millions. 2023 2022 2021 2020 2019 5-year trend; Interest Income: 170,588: 92,807: 57,864: 64,523: 84,040
Get the detailed quarterly/annual income statement for JPMorgan Chase & Co. (JPM). Find out the revenue, expenses and profit or loss over the last fiscal year.
JPMorgan Chase annual net income for 2023 was $47.76B, a 33.07% increase from 2022. JPMorgan Chase annual net income for 2022 was $35.892B, a 22.82% decline from 2021. JPMorgan Chase ann
Using search_text : True


Response(response='The net income in 2019 for JPMorgan Chase bank was $36.4 billion.', source_nodes=[NodeWithScore(node=TextNode(id_='afdf0ed4-fa1b-411e-b6c4-abe7ddc0c7d6', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='ab63a31f-8ea7-459b-b504-311d4eae075d', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='e6ef3b5739a5163dd55bdf0c8a3c63be9aa763d8bdc15752c08295864008902f')}, text='Fiscal year is January-December. All values USD Millions. 2023 2022 2021 2020 2019 5-year trend; Interest Income: 170,588: 92,807: 57,864: 64,523: 84,040\nGet the detailed quarterly/annual income statement for JPMorgan Chase & Co. (JPM). Find out the revenue, expenses and profit or loss over the last fiscal year.\nJPMorgan Chase annual net income for 2023 was $47.76B, a 33.07% increase from 2022. JPMorgan Chase annual net income for 2022 was $35.892B, a 22.82% decline from 2021. JPMorga

In [20]:
response.response

'The net income in 2019 for JPMorgan Chase bank was $36.4 billion.'

## Question 5

I asked 2 questions (the name of the firm and the same question than before):

**Thoughts**:
This kind multiple queries in the same results in the LLM hallucinating ==> Decompose the questions on simple ones.

2019 Net Income in this answer is 48.3 instead of $36.4

In [21]:
prompt = "To which firm corrsponds the financianl report document? and What was the net income in 2019?"
response = corrective_rag.run(prompt, similarity_top_k=2)
response

***************relevancy_results***************
['no', 'yes']



***************relevant_text***************
# Earnings, Diluted Earnings per Share and Return on Tangible Common Equity

$48.3 2004–2022 ($ in billions, except per share and ratio data)

| |Reported|Reported|Reported|Excluding reserve release/build|Excluding reserve release/build|Excluding reserve release/build|
|---|---|---|
| |2020|2021|2022|2020|2021|2022|
|Net income ($B)|$29.1|$48.3|$37.7|$38.4|$39.1|$40.4|
|Diluted EPS ($)|$8.88|$15.36|$12.09|$11.87|$12.35|$12.99|
|ROTCE|14.4%|23.0%|17.7%|19.3%|18.5%|19.1%|

Adjusted net income2 $32.5 $15.36 $10.72 $12.09 $26.9 $29.1 $9.00 $24.4 $24.7 $24.4 $21.7 $8.88 23% $21.3 18% $19.0 $17.4 $17.9 $6.00 $6.19 19% $6.31 $15.4 11% 17% $14.4 15% $14.4 15% 15% 13% $5.19 $5.29 12% $11.7 10% $4.48 $4.34 $8.5 $4.00 $4.33 6% $3.96 $2.35 $5.6 $2.26 $4.5 $1.35 $1.52

2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022

Net income Diluted earnings 

Response(response='The financial report document corresponds to a banking or financial services firm. The net income in 2019 was $48.3 billion.', source_nodes=[NodeWithScore(node=TextNode(id_='2f5be844-c8e5-4efd-be85-1c6d78804263', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='46744819-9a99-43d4-9c30-2f54aebd7497', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='cab101ff79734acc999d802222ca9dcc863b2351bd6f0effd9a1b7e71d29b497')}, text='# Earnings, Diluted Earnings per Share and Return on Tangible Common Equity\n\n$48.3 2004–2022 ($ in billions, except per share and ratio data)\n\n| |Reported|Reported|Reported|Excluding reserve release/build|Excluding reserve release/build|Excluding reserve release/build|\n|---|---|---|\n| |2020|2021|2022|2020|2021|2022|\n|Net income ($B)|$29.1|$48.3|$37.7|$38.4|$39.1|$40.4|\n|Diluted EPS ($)|$8.88|$15.36|$12.09|$11.87|$12.35|$

In [22]:
response.response

'The financial report document corresponds to a banking or financial services firm. The net income in 2019 was $48.3 billion.'